# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

tot_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_train_val.csv')).drop_duplicates()
test_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_test_v0.7.1.csv'))

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
tot_df.columns

Index(['entry_id', 'gender_kw_pred', 'age_kw_pred',
       'affected_groups_level_3_kw', 'gender_snorkel', 'excerpt',
       'analysis_framework_id', 'lead_id', 'project_id', 'verified', 'sectors',
       'subpillars_2d', 'subpillars_1d', 'geo_location',
       'specific_needs_groups', 'severity', 'info_date', 'reliability',
       'affected_groups_level_0', 'affected_groups_level_1',
       'affected_groups_level_2', 'affected_groups_level_3_original',
       'age_original', 'gender_original', 'source_type', 'url', 'website',
       'pillars_1d', 'pillars_2d', 'Casualties', 'Context', 'Covid-19',
       'Displacement', 'Humanitarian Access', 'Information And Communication',
       'Shock/Event', 'At Risk', 'Capacities & Response',
       'Humanitarian Conditions', 'Impact', 'Priority Interventions',
       'Priority Needs', 'affected_groups', 'subpillars', 'secondary_tags',
       'present_age', 'present_gender', 'present_affected_groups_level_3',
       'present_tags'],
      dtype='

In [8]:
"""learning_rates = {
    'present_prim_tags': 5e-5,
    'sectors': 8e-5,
    'pillars_1d': 0.85,
    'pillars_2d': 0.81,
    'subpillars_2d_part1': 0.8, 
   'subpillars_2d_part2': 0.72,
   'subpillars_1d_part1': 0.89,
   'subpillars_1d_part2': 0.85,
   'subpillars_1d_part3': 0.84,
    'gender': 0.84,
    'age': 0.84,
    'specific_needs_groups': 0.79,
    'affected_groups_levels_2_3': 0.99,
    'gender_snorkel': 0.87,
    'subpillars_1d': 0.7,
    'subpillars_2d': 0.7,
    'prim_tags_level1': 0.7,
    'subpillars_second_part': 0.7,
    'subpillars_first_part': 0.7
}"""

"learning_rates = {\n    'present_prim_tags': 5e-5,\n    'sectors': 8e-5,\n    'pillars_1d': 0.85,\n    'pillars_2d': 0.81,\n    'subpillars_2d_part1': 0.8, \n   'subpillars_2d_part2': 0.72,\n   'subpillars_1d_part1': 0.89,\n   'subpillars_1d_part2': 0.85,\n   'subpillars_1d_part3': 0.84,\n    'gender': 0.84,\n    'age': 0.84,\n    'specific_needs_groups': 0.79,\n    'affected_groups_levels_2_3': 0.99,\n    'gender_snorkel': 0.87,\n    'subpillars_1d': 0.7,\n    'subpillars_2d': 0.7,\n    'prim_tags_level1': 0.7,\n    'subpillars_second_part': 0.7,\n    'subpillars_first_part': 0.7\n}"

In [9]:
"""columns = ['excerpt', 'entry_id',
           'present_tags',
           'sectors',
           'pillars_2d',
           'pillars_1d',
           'subpillars_1d_part1',
           'subpillars_1d_part2', 
           'subpillars_1d_part3', 
           'subpillars_2d_part1',
           'subpillars_2d_part2'
        ]"""

"columns = ['excerpt', 'entry_id',\n           'present_tags',\n           'sectors',\n           'pillars_2d',\n           'pillars_1d',\n           'subpillars_1d_part1',\n           'subpillars_1d_part2', \n           'subpillars_1d_part3', \n           'subpillars_2d_part1',\n           'subpillars_2d_part2'\n        ]"

In [10]:
columns = ['excerpt', 'entry_id', 'subpillars', 'sectors', 'secondary_tags']

In [11]:
"""columns = [
    'excerpt', 'entry_id',
    'present_age',
    'present_gender',
    'present_affected_groups_level_3',
    'gender_kw_pred', 
    'age_kw_pred',
       'affected_groups_level_3_kw', 
       'age_original', 
       'gender_original', 
        'affected_groups_level_3_original', 
        'gender_snorkel'
]"""

"columns = [\n    'excerpt', 'entry_id',\n    'present_age',\n    'present_gender',\n    'present_affected_groups_level_3',\n    'gender_kw_pred', \n    'age_kw_pred',\n       'affected_groups_level_3_kw', \n       'age_original', \n       'gender_original', \n        'affected_groups_level_3_original', \n        'gender_snorkel'\n]"

In [12]:

tot_df = tot_df[columns]
test_df = test_df[['excerpt']]

In [13]:
tot_df.columns

Index(['excerpt', 'entry_id', 'subpillars', 'sectors', 'secondary_tags'], dtype='object')

## Sagemaker Prep

### Session

Configure SageMaker

In [14]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [15]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [16]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=50_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [17]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [18]:
from sagemaker.pytorch import PyTorch

instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-all-models-experiments",
    'max_len': 256,
    'epochs': 1,
    'model_name': "microsoft/xtremedistil-l6-h256-uncased",
    'tokenizer_name': "microsoft/xtremedistil-l6-h256-uncased",
    'output_length': 256,
    'learning_rate': 3e-5,
    'training_names':','.join(columns[2:]),
    "instance_type": instance_type,
    'beta_f1': 0.7,
    'nb_repetitions': 1,
    'run_name': 'all_tags_test'
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str(
        '../../../scripts/training/selim/multiclass-lightning/multitask_Tree_like_Architecture'
    ),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [19]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [20]:
# Fit the estimator
estimator.fit(fit_arguments, job_name=job_name)

2022-01-31 11:44:15 Starting - Starting the training job...
2022-01-31 11:44:17 Starting - Launching requested ML instancesProfilerReport-1643629451: InProgress
......
2022-01-31 11:45:34 Starting - Preparing the instances for training.........
2022-01-31 11:47:34 Downloading - Downloading input data
2022-01-31 11:47:34 Training - Downloading the training image.................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-31 11:50:25,288 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-31 11:50:25,310 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-01-31 11:50:28,350 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-01-31 11:50:28,736 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2022-01-31 

      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
      Successfully uninstalled wrapt-1.13.3
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.61.2
    Uninstalling tqdm-4.61.2:
      Successfully uninstalled tqdm-4.61.2
  Attempting uninstall: h5py
    Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
  Attempting uninstall: smdebug
    Found existing installation: smdebug 1.0.9
    Uninstalling smdebug-1.0.9:
      Successfully uninstalled smdebug-1.0.9
  Attempting uninstall: scikit-learn
    Found exi

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2022-01-31 11:53:01.114 algo-1:68 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-01-31 11:53:01.152 algo-1:68 INFO profiler_config_parser.py:102] User has disabled profiler.
[2022-01-31 11:53:01.153 algo-1:68 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2022-01-31 11:53:01.154 algo-1:68 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2022-01-31 11:53:01.154 algo-1:68 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2022-01-31 11:53:01.154 algo-1:68 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2022-01-31 11:53:02.066 algo-1:68 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2022-01-31 11:53:02.066 algo-1:68 INFO hook.py:594] name

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/1563 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/1563 [00:00<?, ?it/s] #015Epoch 0:   3%|▎         | 40/1563 [00:07<04:54,  5.17it/s]#015Epoch 0:   3%|▎         | 40/1563 [00:07<04:54,  5.17it/s, loss=0.0904, v_num=0, val_loss=0.222]#015Epoch 0:   5%|▌         | 80/1563 [00:15<04:41,  5.27it/s, loss=0.0904, v_num=0, val_loss=0.222]#015Epoch 0:   5%|▌         | 80/1563 [00:15<04:41,  5.27it/s, loss=0.0557, v_num=0, val_loss=0.222]#015Epoch 0:   8%|▊         | 120/1563 [00:22<04:29,  5.34it/s, loss=0.0557, v_num=0, val_loss=0.222]#015Epoch 0:   8%|▊         | 120/1563 [00:22<04:29,  5.34it/s, loss=0.047, v_num=0, val_loss=0.222] #015Epoch 0:  10%|█         | 160/1563 [00:29<04:20,  5.38it/s, loss=0.047, v_num=0, val_loss=0.222]#015Epoch 0:  10%|█         | 160/1563 [00:29<04:20,  5.38it/s, loss=0.0413, v_num=0, val_loss=0.222]#015Epoch 0:  13%|█▎     

#015Validating: 100%|██████████| 311/311 [00:18<00:00, 17.24it/s]#033[A#015Epoch 1: 100%|██████████| 1563/1563 [04:12<00:00,  6.19it/s, loss=0.021, v_num=0, val_loss=0.0346, train_loss=0.0224] 
#015                                                             #033[A#015Epoch 1:   0%|          | 0/1563 [00:00<?, ?it/s, loss=0.021, v_num=0, val_loss=0.0346, train_loss=0.0224]           #015Epoch 2:   0%|          | 0/1563 [00:00<?, ?it/s, loss=0.021, v_num=0, val_loss=0.0346, train_loss=0.0224]#015Epoch 2:   3%|▎         | 40/1563 [00:07<04:53,  5.20it/s, loss=0.021, v_num=0, val_loss=0.0346, train_loss=0.0224]#015Epoch 2:   3%|▎         | 40/1563 [00:07<04:53,  5.20it/s, loss=0.02, v_num=0, val_loss=0.0346, train_loss=0.0224] #015Epoch 2:   5%|▌         | 80/1563 [00:15<04:42,  5.25it/s, loss=0.02, v_num=0, val_loss=0.0346, train_loss=0.0224]#015Epoch 2:   5%|▌         | 80/1563 [00:15<04:42,  5.25it/s, loss=0.0206, v_num=0, val_loss=0.0346, train_loss=0.0224]#015Epoch 2:   8%|▊         

...

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/1564 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/1564 [00:00<?, ?it/s] #015Epoch 0:   3%|▎         | 40/1564 [00:07<04:28,  5.69it/s]#015Epoch 0:   3%|▎         | 40/1564 [00:07<04:28,  5.69it/s, loss=0.0767, v_num=0, val_loss=0.202]#015Epoch 0:   5%|▌         | 80/1564 [00:13<04:13,  5.84it/s, loss=0.0767, v_num=0, val_loss=0.202]#015Epoch 0:   5%|▌         | 80/1564 [00:13<04:13,  5.84it/s, loss=0.051, v_num=0, val_loss=0.202] #015Epoch 0:   8%|▊         | 120/1564 [00:20<04:07,  5.83it/s, loss=0.051, v_num=0, val_loss=0.202]#015Epoch 0:   8%|▊         | 120/1564 [00:20<04:07,  5.83it/s, loss=0.0468, v_num=0, val_loss=0.202]#015Epoch 0:  10%|█         | 160/1564 [00:27<03:59,  5.86it/s, loss=0.0468, v_num=0, val_loss=0.202]#015Epoch 0: 

#015                                                             #033[A#015Epoch 0:   0%|          | 0/1564 [00:00<?, ?it/s, loss=0.0277, v_num=0, val_loss=0.0419, train_loss=0.0366]           #015Epoch 1:   0%|          | 0/1564 [00:00<?, ?it/s, loss=0.0277, v_num=0, val_loss=0.0419, train_loss=0.0366]#015Epoch 1:   3%|▎         | 40/1564 [00:07<04:29,  5.66it/s, loss=0.0277, v_num=0, val_loss=0.0419, train_loss=0.0366]#015Epoch 1:   3%|▎         | 40/1564 [00:07<04:29,  5.66it/s, loss=0.0262, v_num=0, val_loss=0.0419, train_loss=0.0366]#015Epoch 1:   5%|▌         | 80/1564 [00:13<04:15,  5.81it/s, loss=0.0262, v_num=0, val_loss=0.0419, train_loss=0.0366]#015Epoch 1:   5%|▌         | 80/1564 [00:13<04:15,  5.81it/s, loss=0.027, v_num=0, val_loss=0.0419, train_loss=0.0366] #015Epoch 1:   8%|▊         | 120/1564 [00:20<04:06,  5.86it/s, loss=0.027, v_num=0, val_loss=0.0419, train_loss=0.0366]#015Epoch 1:   8%|▊         | 120/1564 [00:20<04:06,  5.86it/s, loss=0.0243, v_num=0, val_loss=0

#015Validating: 100%|██████████| 290/290 [00:14<00:00, 19.97it/s]#033[A#015Epoch 1: 100%|██████████| 1564/1564 [03:52<00:00,  6.73it/s, loss=0.0231, v_num=0, val_loss=0.0386, train_loss=0.024] 

2022-01-31 13:05:55 Stopping - Stopping the training job#015                                                             #033[A#015Epoch 1:   0%|          | 0/1564 [00:00<?, ?it/s, loss=0.0231, v_num=0, val_loss=0.0386, train_loss=0.024]           #015Epoch 2:   0%|          | 0/1564 [00:00<?, ?it/s, loss=0.0231, v_num=0, val_loss=0.0386, train_loss=0.024]#015Epoch 2:   3%|▎         | 40/1564 [00:07<04:34,  5.55it/s, loss=0.0231, v_num=0, val_loss=0.0386, train_loss=0.024]#015Epoch 2:   3%|▎         | 40/1564 [00:07<04:34,  5.55it/s, loss=0.0236, v_num=0, val_loss=0.0386, train_loss=0.024]#015Epoch 2:   5%|▌         | 80/1564 [00:14<04:21,  5.67it/s, loss=0.0236, v_num=0, val_loss=0.0386, train_loss=0.024]#015Epoch 2:   5%|▌         | 80/1564 [00:14<04:21,  5.67it/s, loss=0.0224, v_num=0, val_l

Job ended with status 'Stopped' rather than 'Completed'. This could mean the job timed out or stopped early for some other reason: Consider checking whether it completed as you expect.


Training seconds: 4907
Billable seconds: 4907
